# Clase del Juego

In [1]:
import numpy as np
from IPython.display import clear_output
from random import randrange, choice
class Game():
    def __init__(self,colum,fil):
        self.mesa= np.zeros( (fil,colum ) )
        self.columnas=colum-1
        self.filas=fil-1
        self.end=0
        self.mov=0
        
    def movimiento(self,colum,valor):
        x=self.filas
        y=colum
        celda=self.mesa[x-1][y]
        while(celda==0 and (x-1)>=0):
            x=x-1
            celda=self.mesa[x-1][y]
        self.mesa[x][y]=valor
        return x
    
    def verifica_multiplos(self,x,y):
        valor= self.mesa[x][y]
        suma=valor
        #Celda de arriba
        if(x-1>=0):
            if(self.mesa[x-1][y]==valor):
                suma=suma+self.mesa[x-1][y]
                self.mesa[x-1][y]=0.0
        #Celda de Abajo
        if(x+1<=self.filas):
            if(self.mesa[x+1][y]==valor):
                suma=suma+self.mesa[x+1][y]
                self.mesa[x+1][y]=0.0
        #Celda izquierda
        if(y-1>=0):
            if(self.mesa[x][y-1]==valor):
                suma=suma+self.mesa[x][y-1]
                self.mesa[x][y-1]=0.0
        #Celda derecha
        if(y+1<=self.columnas):
            if(self.mesa[x][y+1]==valor):
                suma=suma+self.mesa[x][y+1]
                self.mesa[x][y+1]=0.0
        if(valor!=suma):
            self.mesa[x][y]=suma
            self.verifica_multiplos(x,y)
            
        self.verificar_espacios()
        
    def verificar_espacios(self):
        for x in range(self.filas+1):
            for y in range(self.columnas+1):
                if(x+1<=self.filas):
                    if(self.mesa[x][y]==0.0 and self.mesa[x+1][y]!=0.0):
                        self.mesa[x][y]=self.mesa[x+1][y]
                        self.mesa[x+1][y]=0.0
                        self.verifica_multiplos(x,y)
        self.verificar_end()
    
    def verificar_end(self):
        for y in range(self.columnas+1):
            if(self.mesa[self.filas][y]!=0.0):
                self.end=1
                
    def run(self):
        
        while(self.end==0):
            clear_output()
            print(self.mesa)
            valor= choice([2.0,4.0,8.0,16.0,32.0])
            print(valor)
            y=int(input())
            x= self.movimiento(y,valor)
            self.verifica_multiplos(x,y)
        print(self.mesa)
                
       
    
    
    

In [2]:
game= Game(3,7)
game.run()

[[  8.   0.   0.]
 [  2.   0.   0.]
 [  4.   0.   0.]
 [ 16.   0.   0.]
 [  2.   0.   0.]
 [ 16.   0.   0.]
 [  0.   0.   0.]]
4.0
0
[[  8.   0.   0.]
 [  2.   0.   0.]
 [  4.   0.   0.]
 [ 16.   0.   0.]
 [  2.   0.   0.]
 [ 16.   0.   0.]
 [  4.   0.   0.]]


# Código de la Red Neuronal

In [2]:
import numpy as np
import math
#""" Clase que representa la capa de un perceptron sigmoide"""
class Perceptron: 
    def __init__(self):
        self.pesos= [] # """Arreglo donde se guardan los pesos"""
        self.datos=[]  #""" Arreglo donde se guardan los inputs"""
        self.b=0  #"""Este valor representa el bais"""
        self.out=0 #""" Este valor represente el output del sigmoide"""
        self.error=0 #""" El error """
        self.delta=0  #""" El delta"""
        self.lr=0.1 #""" El learning rata"""
        self.bout=0  # """Este valor represente en binario el output, por si se necesita"""
    
   # """ Método para agregar el learning rate"""
    def agregarlr(self,valor):
        self.lr=valor
    #""" Agregar pesos"""
    def agregarpeso(self,peso):
        self.pesos.append(peso)
    #""" Agregar los inputs"""    
    def agregardato(self,dato):
        self.datos.append(dato)
    #""" Agregar bais"""    
    def agregarbais(self,bais):
        self.b=bais
     #"""Agregar erro, este se utiliza si este perectron está en la última capa """"
      #   """ya que aquí el erro es la resta del output con el output esperado """   
    def agregarerror(self,valor):
        self.error=valor
   # """ Calcular el output del sigmoide, ya debe estar ingresados lo datos y los pesos"""
    def run(self):
        ps=np.array(self.pesos)
        dt=np.array(self.datos)
        x= ps*dt
        suma=x.sum()
        exponente = -1*(suma+self.b)
        denominador = 1 + np.exp(exponente)
        self.out=1/denominador
        if self.out>=0.5: #"""Criterio para calcular el output binario"""
            self.bout=1
        else:
            self.bout=0
    #"""Agregar datos"""
    def agregardatos(self,dats):
        self.datos=dats
     #"""Vaciar datos"""   
    def vaciardatos(self):
        self.datos=[]
        
    #"""Calcular error, ingresando los delta de la capa siguiente"""
    def calcularerror(self,deltasnextcapa,pesosnextcapa):
        ps=np.array(pesosnextcapa)
        delt=np.array(deltasnextcapa)
        
        mult=ps*delt
        self.error=mult.sum()
    #"""Calcular del delta"""    
    def calculardelta(self):
        self.delta=self.error*(self.out*(1-self.out))
    
    #Despues de tener el delta se realiza el ajuste de los pesos, este se usa
    #para el paso 3 de la backpropagation. 
    def ajustarpesos(self):
        nuevospesos=[]
        x=0
        for peso in self.pesos:
            nuevospesos.append(peso+(self.lr*self.delta*self.datos[x]))
            x=x+1
        self.pesos=nuevospesos
        self.b=self.b + self.lr*self.delta    

In [3]:
class Capa:
    #"""Clase que representa una capa de red neuronal"""
    def __init__(self):
        self.perceptrones=[] #"""Arreglo con los pereceptrones"""
        self.input=[] #"""Inputs que se le pasaran a  los pereceptrones """
        self.output=[] # """Outputs que regresaran los perceptrones"""
        self.deltas=[]  #"""Deltas de cada uno de los perceptrones """
        self.siguientecapa = None # """Siguiente capa"""
        self.anteriorcapa =None  # """Capa anterior"""
        self.boutput=[]
        self.pesos=[]  #Arreglo que guarda los pesos de cada perceptron, arreglo de arreglos
    def getnextcapa(self, capa):
        self.siguientecapa = capa
    
    def getpreviouscapa(self,capa):
        self.anteriorcapa = capa
   # """Agregar inputs"""
    def getinput(self,datos):
        self.input=datos
    # Agregar perceptron individual
    def getperceptron(self, perceptron):
        self.perceptrones.append(perceptron)
        self.pesos.append(perceptron.pesos)
   
    #Cargar los pesos de los perceptrones
    def cargarpesos(self):
        pesos=[]
        for per in self.perceptrones:
            pesos.append(per.pesos)
        self.pesos=pesos
    # Agregar perceptrones arreglo     
    def setperceptrones(self, arreglo):
        self.perceptrones=arreglo
        pesos=[]
        for per in arreglo:
            pesos.append(per.pesos)
        self.pesos=pesos
        
   #"""Recorrido para calcular output """
    def Feeding(self):
        self.output=[]
        self.boutput=[]
        for perc in self.perceptrones: #"""Cada perceptron recibe los datos y su respuestas son cargadas al outputs"""
            perc.agregardatos(self.input)
            perc.run()
            self.output.append(perc.out)
            self.boutput.append(perc.bout)
        if self.siguientecapa != None: #"""Si no es la última capa, se pasá mi output como input de la siguiente capa"""
            self.siguientecapa.getinput(self.output)
            self.siguientecapa.Feeding()
    
   #Método backprogation para la última cap, recibe un arreglo con los output esperados

    def backpropagation(self, expectoutput):
        self.deltas=[] #Vaciamo los deltas
        x=0
        for perc in self.perceptrones:
            perc.agregarerror(expectoutput[x]-perc.out)# """Cada pereceptron recibe su error"""
            perc.calculardelta()#"""Cada perceptron calcula su delta"""
            self.deltas.append(perc.delta) #"""Se agregan a los delta de la capaa """
            x=x+1
        self.anteriorcapa.backpropagation1(self.deltas,self.pesos) #Backpropagation para otreas capas. 
        #Despues se propaga el error, comenzamos a ajustar los pesos.
        for perc in self.perceptrones:
            perc.ajustarpesos()
        self.cargarpesos()
    #Métod backpropagation para capas distan a las final,recibe los deltas y pesos de la sigueinte capa.     
    def backpropagation1(self,deltas,pesos):
        self.deltas=[] #Vaciamos los deltas
        x=0
        for perc in self.perceptrones:
            pecorres=[] #Pesos correspondientes para un perceptron específico
            for pp in pesos: #pp es un arreglo con los pesos de un perceptron de la capa siguiente
                pecorres=pp[x]     #Sacamos nuestro peso correspondiente de ese perceotron
            perc.calcularerror(deltas,pecorres) # El perecptron calcula el erro
            perc.calculardelta() # El perceptron calcula el delta.
            self.deltas.append(perc.delta) #Agregamos el delta al arreglo de la capa
            x=x+1
        if(self.anteriorcapa==None): #Si es la primera capa, detemos la propagación
            for perc in self.perceptrones: 
                perc.ajustarpesos() # Paso 3 de el backpropagation, ajustamos los pesos
            self.cargarpesos() #Cargamos lo pesos al arreglo de la capa
        else:
            self.anteriorcapa.backpropagation1(self.deltas,self.pesos) #Seguimos con la propagación
            for perc in self.perceptrones:
                perc.ajustarpesos() #Al terminar la propagación ajustamos los pesos.
            self.cargarpesos()
        
        

In [4]:

class Fabrica:
    def __init__(self):
        self.lr=0.1 #learning range con el que se crearn los perceptrones
        self.bais=-1 #bais con el que se crearan los peceptrones
        
    def getlr(self,valor):
        self.lr=valor
        
    def getbais(self,valor):
        self.bais=valor
    #Método que crea un perecpetron con cantidad especifica de pesos   
    def crearPerceptron(self,cantPesos):
        per= Perceptron()
        for x in range(cantPesos):
            per.agregarpeso(uniform(-2,2))
        per.agregarlr(self.lr)
        per.agregarbais(self.bais)
        return per
    #Método que crea una capa con una cantidad especifica de perceptrones y pesos.
    def crearCapa(self, cantPer, cantPesos):
        cap= Capa()
        for x in range(cantPer):
            cap.getperceptron(self.crearPerceptron(cantPesos))
        return cap    
    
    

In [5]:
class Red_Neuronal:
    def __init__(self):
        self.capinicial=None #Capa inicial 
        self.capintemedias=[] #Arreglo donde se guardaran las capas intemedias, deben ser ingresadas en orden.
        self.capfinal=None
    #Se agrega la capa incial a la red neuronal, se asume que esta tiene los perceptrones y pesos correctos.
    def setcapinicial(self,capa):
        self.capinicial=capa
    #Agregar capa final.
    def setcapfinal(self, capa):
        self.capfinal= capa
    #Agregar una capa intemedia, se debe agregar en orden
    def setcapaintermedia(self, capa):
        self.capintemedias.append(capa)
    #Método para que cada capa reciba su capa anterior y su capa siguiente    
    def enlazar(self):
        if len(self.capintemedias)==0: #Si no hay capas intermedias
            self.capinicial.getnextcapa(self.capfinal)   #Se enlazan la primera con la última
            self.capfinal.getpreviouscapa(self.capinicial)
        else:  #Si hay capas intemedias
            x=0
            self.capinicial.getnextcapa(self.capintemedias[x]) #Se enlaza la capa inicial, con la primera capa del arreglo
            while x<(len(self.capintemedias)-1):
                self.capintemedias[x].getnextcapa(self.capintemedias[x+1]) #Se enlaza cada capa, con su siguiente
                x=x+1
            x=len(self.capintemedias)-1
            while x>0:
                self.capintemedias[x].getpreviouscapa(self.capintemedias[x-1])#Se enlaza cada capa, con la anterior
                x=x-1
            x=len(self.capintemedias)-1
            self.capfinal.getpreviouscapa(self.capintemedias[x]) #Se enlza la  útima capa del arrglo, con la capa final
            self.capintemedias[x].getnextcapa(self.capfinal)
    
    def comparador(self,conjunto1,conjunto2):
        x=1
        i=0
        for z in conjunto1:
            if(z!=conjunto2[i]):
                x=0
                return x
            i=i+1
        return x
    
    #Método para entrenar, devuelve los errores
    def entrenar(self,settrain,num, settest):
        inputrain=settrain[0]
        outputtrain=settrain[1]
        inputtest=settest[0]
        outputtest=settest[1]
        intentos=[]
        resultados=[]
        
        for x in range(num):
            i=0
            for tupla in inputrain:
                self.capinicial.getinput(tupla)
                self.capinicial.Feeding()
                self.capfinal.backpropagation(outputtrain[i])
                i=i+1
            i=0
            errores=0
            for tupla in inputtest:
                self.capinicial.getinput(tupla)
                self.capinicial.Feeding()
                if self.comparador(self.capfinal.boutput,outputtest[i])==0:
                    errores=errores+1
                i=i+1
            resultados.append(errores)
            intentos.append(x+1)
        return [intentos,resultados]

# Código Algóritmo Genético 

In [6]:
from random import random, randrange 
class Genetic_Algoritmo:
    def __init__(self):
        self.Mutation_Rate=0.01
        self.Genes=0
        self.Population=[]
        self.Evaluation=[]
        self.Generador=None
        self.Fitness=None
        self.Selecction=[]
    
    def genera_popinicial(self, num):
        for x in range(num):
            indiv=[]
            for w in range(self.Genes):
                indiv.append(self.Generador(w))
            self.Population.append(indiv)
    
    def Evaluate(self):
        self.Evaluation=[]
        for x in range(len(self.Population)):
            self.Evaluation.append(self.Fitness(self.Population[x]))
    
    def Tournament_selection(self,k):
        ind=-1;
        for x in range(k):
            r= randrange(len(self.Population)) 
            if(ind==-1 or self.Evaluation[r]>self.Evaluation[ind]):
                ind=r
        self.Selecction.append(self.Population[ind])
        
    def Reproduction(self):
        largo=len(self.Population)
        newPop=[]
        self.Selecction=[]
        for x in range(2*largo):
            self.Tournament_selection(5)
        for x in range(largo):
            newindv=[]
            indiv1= self.Selecction[x]
            indiv2= self.Selecction[x+largo]
            point_mixing= randrange(self.Genes)
            for x in range(self.Genes):
                if(x>point_mixing):
                    newindv.append(indiv2[x])
                else:
                    newindv.append(indiv1[x])
            i=0
            for gen in newindv:
                rand= random()
                if rand<self.Mutation_Rate:
                    newgen=self.Generador(i)
                    newindv[i]=newgen
                i=i+1
            newPop.append(newindv)
        self.Population=newPop

In [7]:
from random import randrange, choice,uniform
game= Game(3,7)
fabri= Fabrica()
def normalizar(arreglo):
    salida=[]
    for x in arreglo:
        salida.append(x/1024)
    return salida

def Generador(ind):
    if(ind<25):
        return fabri.crearPerceptron(22)
    else:
        return fabri.crearPerceptron(25)
    
def MaxValoMesa(mesa):
    maxx=0;
    for fila in mesa:
        for valor in fila:
            if (valor>maxx):
                maxx=valor
    return maxx
    
def Evaluador(indv):
    game= Game(3,7)
    acapa1=[]
    acapa2=[]
    for x in range(28):
        if(x<25):
            acapa1.append(indv[x])
        else:
            acapa2.append(indv[x])
    Capa1= Capa()
    Capa2=Capa()
    Capa1.setperceptrones(acapa1)
    Capa2.setperceptrones(acapa2)
    red1= Red_Neuronal()
    red1.setcapinicial(Capa1) #Capa incial.
    red1.setcapfinal(Capa2) #Capa final
    red1.enlazar()
    mov=0
    while(game.end==0):
        valor= choice([2.0,4.0,8.0,16.0,32.0])
        entrada=[]
        entrada.append(valor)
        for fila in game.mesa:
            for celda in fila:
                entrada.append(celda)
        red1.capinicial.getinput(normalizar(entrada))
        red1.capinicial.Feeding()
        
        outp=red1.capfinal.output
        i=0
        maxx=0
        i2=0
        for xot in outp:
            if(maxx<=xot):
                i2=i
                maxx=xot
            i=i+1
        x= game.movimiento(i2,valor)
        game.verifica_multiplos(x,i2)
        #print(game.mesa)
        mov=mov+1
    return mov

def verjugar(indv):
    game= Game(3,7)
    print("ss")
    acapa1=[]
    acapa2=[]
    for x in range(28):
        if(x<25):
            acapa1.append(indv[x])
        else:
            acapa2.append(indv[x])
    Capa1= Capa()
    Capa2= Capa()
    Capa1.setperceptrones(acapa1)
    Capa2.setperceptrones(acapa2)
    red1= Red_Neuronal()
    red1.setcapinicial(Capa1) #Capa incial.
    red1.setcapfinal(Capa2) #Capa final
    red1.enlazar()
    mov=0
    while(game.end==0):
        valor= choice([2.0,4.0,8.0,16.0,32.0])
        entrada=[]
        entrada.append(valor)
        for fila in game.mesa:
            for celda in fila:
                entrada.append(celda)
        red1.capinicial.getinput(normalizar(entrada))
        red1.capinicial.Feeding()
        
        outp=red1.capfinal.output
        i=0
        maxx=0
        i2=0
        for xot in outp:
            if(maxx<=xot):
                i2=i
                maxx=xot
            i=i+1
        x= game.movimiento(i2,valor)
        game.verifica_multiplos(x,i2)
        print(game.mesa)
        mov=mov+1
        print(mov)
Genetic1 = Genetic_Algoritmo()
Genetic1.Fitness=Evaluador
Genetic1.Generador=Generador
Genetic1.Genes=28
Genetic1.genera_popinicial(1000)
Genetic1.Evaluate()

In [8]:
print(Genetic1.Evaluation)

[10, 13, 8, 9, 8, 9, 8, 11, 17, 9, 9, 9, 8, 7, 8, 7, 7, 12, 8, 8, 8, 10, 8, 7, 10, 8, 13, 7, 7, 8, 7, 8, 13, 8, 8, 10, 9, 9, 11, 9, 8, 7, 8, 9, 8, 8, 7, 8, 12, 7, 10, 12, 7, 8, 8, 8, 13, 10, 7, 7, 8, 12, 7, 9, 8, 8, 9, 9, 7, 8, 8, 11, 8, 10, 9, 8, 8, 7, 8, 7, 7, 10, 8, 7, 8, 7, 8, 7, 7, 7, 7, 9, 7, 9, 8, 8, 8, 8, 8, 7, 11, 7, 12, 11, 15, 8, 8, 8, 7, 7, 10, 12, 10, 7, 9, 8, 9, 8, 12, 9, 8, 9, 10, 8, 7, 8, 10, 10, 7, 7, 7, 8, 8, 10, 11, 11, 10, 9, 9, 11, 7, 7, 10, 8, 10, 7, 17, 7, 10, 8, 8, 10, 9, 11, 7, 8, 8, 8, 10, 7, 9, 8, 8, 8, 7, 7, 10, 9, 7, 8, 7, 8, 9, 13, 7, 8, 7, 9, 7, 8, 9, 7, 12, 8, 9, 12, 9, 8, 9, 8, 8, 9, 9, 7, 8, 9, 8, 7, 9, 8, 8, 11, 9, 7, 9, 9, 8, 7, 8, 9, 8, 9, 8, 7, 7, 7, 8, 7, 13, 9, 8, 8, 7, 10, 7, 11, 10, 10, 7, 7, 10, 8, 7, 9, 9, 8, 9, 17, 7, 9, 7, 8, 7, 9, 8, 12, 7, 9, 8, 10, 8, 10, 7, 7, 8, 12, 7, 10, 11, 7, 7, 8, 8, 10, 8, 7, 7, 8, 10, 7, 10, 7, 8, 7, 8, 10, 10, 10, 8, 9, 8, 11, 11, 7, 10, 7, 7, 8, 8, 7, 9, 7, 13, 10, 9, 9, 8, 12, 8, 8, 10, 11, 7, 8, 7, 13, 12, 8

In [9]:
for x in range(200):
    Genetic1.Reproduction()
    Genetic1.Evaluate()
print(Genetic1.Evaluation)

[21, 64, 41, 34, 29, 28, 31, 40, 31, 40, 35, 24, 31, 20, 24, 33, 29, 43, 44, 29, 31, 16, 14, 39, 33, 31, 33, 47, 37, 31, 11, 44, 25, 32, 41, 20, 27, 36, 25, 25, 39, 36, 40, 28, 43, 13, 36, 29, 27, 39, 29, 13, 27, 25, 31, 35, 68, 9, 32, 16, 32, 23, 29, 27, 33, 25, 45, 25, 29, 36, 31, 34, 22, 32, 37, 39, 33, 45, 11, 47, 18, 24, 22, 33, 47, 8, 26, 48, 43, 26, 32, 36, 33, 36, 52, 26, 48, 41, 22, 40, 34, 18, 44, 28, 46, 16, 17, 25, 17, 22, 27, 44, 17, 35, 22, 23, 28, 33, 36, 13, 30, 34, 30, 29, 23, 47, 41, 34, 36, 34, 45, 21, 27, 36, 31, 21, 45, 31, 33, 41, 40, 15, 31, 48, 39, 37, 7, 23, 16, 11, 24, 22, 14, 34, 31, 20, 33, 40, 35, 23, 22, 31, 26, 47, 25, 33, 32, 21, 39, 26, 30, 32, 27, 28, 40, 30, 27, 39, 39, 28, 42, 36, 18, 33, 35, 38, 34, 41, 25, 29, 44, 29, 30, 41, 17, 32, 14, 29, 39, 34, 24, 26, 25, 21, 49, 26, 25, 39, 27, 26, 39, 32, 21, 30, 31, 45, 24, 36, 45, 50, 44, 32, 34, 19, 44, 33, 10, 54, 36, 21, 28, 27, 23, 42, 13, 17, 30, 35, 32, 36, 20, 26, 27, 19, 55, 33, 40, 13, 24, 16, 38

In [19]:
def buscarmax(lista):
    i=0
    maxx=0
    for x in range(len(lista)):
        if (lista[x]>maxx):
            maxx=lista[x]
            i=x
    print(i,maxx)
    return i
mejor= buscarmax(Genetic1.Evaluation)

verjugar(Genetic1.Population[mejor])

56 68
ss
[[ 16.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
1
[[ 16.   4.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
2
[[ 16.   8.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
3
[[ 16.   8.   0.]
 [  0.   2.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
4
[[ 16.   8.   0.]
 [  8.   2.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
5
[[ 16.   8.   0.]
 [  8.   2.   0.]
 [  0.   4.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
6
[[ 32.   8.   0.]
 [  0.   2.   0.]
 [  0.   4.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
7
[[ 64.   8.   0.]
 [  0.   2.   0.]
 [  0.   4.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.

In [20]:
def Evaluador1(indv):
    game= Game(3,7)
    acapa1=[]
    acapa2=[]
    for x in range(28):
        if(x<25):
            acapa1.append(indv[x])
        else:
            acapa2.append(indv[x])
    Capa1= Capa()
    Capa2=Capa()
    Capa1.setperceptrones(acapa1)
    Capa2.setperceptrones(acapa2)
    red1= Red_Neuronal()
    red1.setcapinicial(Capa1) #Capa incial.
    red1.setcapfinal(Capa2) #Capa final
    red1.enlazar()
    mov=0
    while(game.end==0):
        valor= choice([2.0,4.0,8.0,16.0,32.0])
        entrada=[]
        entrada.append(valor)
        for fila in game.mesa:
            for celda in fila:
                entrada.append(celda)
        red1.capinicial.getinput(normalizar(entrada))
        red1.capinicial.Feeding()
        
        outp=red1.capfinal.output
        i=0
        maxx=0
        i2=0
        for xot in outp:
            if(maxx<=xot):
                i2=i
                maxx=xot
            i=i+1
        x= game.movimiento(i2,valor)
        game.verifica_multiplos(x,i2)
        #print(game.mesa)
        mov=mov+1
    puntaje= MaxValoMesa(game.mesa)
    return puntaje

Genetic1.Fitness=Evaluador1
Genetic1.Evaluate()
print(Genetic1.Evaluation)

[128.0, 128.0, 64.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 256.0, 256.0, 192.0, 64.0, 64.0, 128.0, 256.0, 128.0, 128.0, 512.0, 128.0, 96.0, 96.0, 256.0, 64.0, 64.0, 256.0, 128.0, 128.0, 128.0, 32.0, 32.0, 128.0, 256.0, 128.0, 192.0, 128.0, 128.0, 256.0, 64.0, 128.0, 128.0, 64.0, 128.0, 96.0, 256.0, 128.0, 128.0, 64.0, 128.0, 96.0, 128.0, 64.0, 128.0, 128.0, 128.0, 64.0, 256.0, 32.0, 64.0, 96.0, 256.0, 64.0, 64.0, 64.0, 256.0, 128.0, 64.0, 128.0, 64.0, 128.0, 128.0, 256.0, 128.0, 96.0, 64.0, 128.0, 64.0, 192.0, 128.0, 256.0, 256.0, 96.0, 192.0, 128.0, 128.0, 32.0, 192.0, 128.0, 64.0, 192.0, 128.0, 96.0, 192.0, 128.0, 64.0, 128.0, 128.0, 32.0, 128.0, 64.0, 128.0, 128.0, 256.0, 64.0, 128.0, 64.0, 256.0, 128.0, 128.0, 256.0, 64.0, 256.0, 128.0, 256.0, 64.0, 128.0, 192.0, 128.0, 128.0, 64.0, 128.0, 192.0, 192.0, 256.0, 128.0, 128.0, 256.0, 128.0, 128.0, 128.0, 64.0, 128.0, 192.0, 128.0, 128.0, 128.0, 256.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 128.0, 64.0, 64.0, 32.0, 64.0, 128.0, 

In [21]:
for x in range(200):
    Genetic1.Reproduction()
    Genetic1.Evaluate()
print(Genetic1.Evaluation)

[128.0, 128.0, 128.0, 128.0, 256.0, 96.0, 128.0, 128.0, 128.0, 96.0, 128.0, 128.0, 128.0, 128.0, 256.0, 256.0, 256.0, 256.0, 64.0, 256.0, 64.0, 128.0, 256.0, 64.0, 32.0, 128.0, 256.0, 256.0, 128.0, 256.0, 64.0, 192.0, 64.0, 192.0, 128.0, 192.0, 32.0, 64.0, 128.0, 128.0, 128.0, 128.0, 256.0, 128.0, 128.0, 128.0, 128.0, 64.0, 64.0, 192.0, 64.0, 96.0, 64.0, 64.0, 256.0, 64.0, 96.0, 64.0, 128.0, 128.0, 128.0, 128.0, 256.0, 256.0, 128.0, 128.0, 192.0, 128.0, 256.0, 256.0, 128.0, 128.0, 64.0, 128.0, 128.0, 64.0, 128.0, 128.0, 128.0, 128.0, 64.0, 64.0, 256.0, 96.0, 128.0, 128.0, 128.0, 256.0, 128.0, 128.0, 64.0, 128.0, 128.0, 128.0, 128.0, 64.0, 128.0, 128.0, 32.0, 128.0, 128.0, 96.0, 128.0, 128.0, 128.0, 256.0, 128.0, 128.0, 128.0, 256.0, 64.0, 64.0, 128.0, 256.0, 128.0, 192.0, 128.0, 96.0, 64.0, 256.0, 64.0, 128.0, 128.0, 128.0, 128.0, 192.0, 128.0, 128.0, 128.0, 64.0, 128.0, 128.0, 128.0, 64.0, 128.0, 128.0, 256.0, 64.0, 64.0, 128.0, 128.0, 128.0, 128.0, 128.0, 256.0, 96.0, 256.0, 128.0, 9

In [35]:
def buscarmax(lista):
    i=0
    maxx=0
    for x in range(len(lista)):
        if (lista[x]>maxx):
            maxx=lista[x]
            i=x
    print(i,maxx)
    return i
mejor= buscarmax(Genetic1.Evaluation)

verjugar(Genetic1.Population[mejor])

278 384.0
ss
[[ 8.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
[[ 16.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
2
[[ 16.   0.   0.]
 [ 32.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
3
[[ 16.   0.   0.]
 [ 32.   0.   0.]
 [ 16.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
4
[[ 16.   0.   0.]
 [ 32.   0.   0.]
 [ 16.   0.   0.]
 [ 32.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
5
[[ 16.   4.   0.]
 [ 32.   0.   0.]
 [ 16.   0.   0.]
 [ 32.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
6
[[ 16.   4.   2.]
 [ 32.   0.   0.]
 [ 16.   0.   0.]
 [ 32.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
7
[[ 16.   4.   2.]
 [ 32.   0.   0.]
 [ 16.   0.   0.]
 [ 64.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0

In [36]:
def Evaluador2(indv):
    game= Game(3,7)
    acapa1=[]
    acapa2=[]
    for x in range(28):
        if(x<25):
            acapa1.append(indv[x])
        else:
            acapa2.append(indv[x])
    Capa1= Capa()
    Capa2=Capa()
    Capa1.setperceptrones(acapa1)
    Capa2.setperceptrones(acapa2)
    red1= Red_Neuronal()
    red1.setcapinicial(Capa1) #Capa incial.
    red1.setcapfinal(Capa2) #Capa final
    red1.enlazar()
    mov=0
    while(game.end==0):
        valor= choice([2.0,4.0,8.0,16.0,32.0])
        entrada=[]
        entrada.append(valor)
        for fila in game.mesa:
            for celda in fila:
                entrada.append(celda)
        red1.capinicial.getinput(normalizar(entrada))
        red1.capinicial.Feeding()
        
        outp=red1.capfinal.output
        i=0
        maxx=0
        i2=0
        for xot in outp:
            if(maxx<=xot):
                i2=i
                maxx=xot
            i=i+1
        x= game.movimiento(i2,valor)
        game.verifica_multiplos(x,i2)
        #print(game.mesa)
        mov=mov+1
    puntaje= MaxValoMesa(game.mesa)
    return puntaje*mov

Genetic1.Fitness=Evaluador2
Genetic1.Evaluate()
print(Genetic1.Evaluation)

[960.0, 1728.0, 8960.0, 2016.0, 4224.0, 1664.0, 4736.0, 15104.0, 11264.0, 8448.0, 1216.0, 1472.0, 14592.0, 3840.0, 832.0, 1664.0, 2880.0, 3936.0, 960.0, 4736.0, 3712.0, 3200.0, 3712.0, 4224.0, 1728.0, 896.0, 10240.0, 8448.0, 2432.0, 1920.0, 4864.0, 3584.0, 2304.0, 1152.0, 1472.0, 2944.0, 9728.0, 928.0, 3648.0, 896.0, 4992.0, 896.0, 1728.0, 3200.0, 4480.0, 3200.0, 5504.0, 9216.0, 1472.0, 512.0, 1216.0, 3072.0, 864.0, 2304.0, 5120.0, 1216.0, 3200.0, 1088.0, 7680.0, 5120.0, 2048.0, 1984.0, 1792.0, 2048.0, 6400.0, 3584.0, 1536.0, 1472.0, 1984.0, 1472.0, 3360.0, 6912.0, 4608.0, 7680.0, 7424.0, 4736.0, 2688.0, 3072.0, 3840.0, 2176.0, 2688.0, 4352.0, 6144.0, 4864.0, 3584.0, 3968.0, 640.0, 3168.0, 224.0, 576.0, 2688.0, 8960.0, 3328.0, 6144.0, 3584.0, 2496.0, 4992.0, 11520.0, 1408.0, 1088.0, 8704.0, 3840.0, 1216.0, 3072.0, 4864.0, 6016.0, 2688.0, 10240.0, 4352.0, 3072.0, 4096.0, 1408.0, 3456.0, 1792.0, 12032.0, 4992.0, 3200.0, 6912.0, 3072.0, 4736.0, 2944.0, 5248.0, 3200.0, 1664.0, 9728.0, 5760

In [37]:
for x in range(200):
    Genetic1.Reproduction()
    Genetic1.Evaluate()
print(Genetic1.Evaluation)

KeyboardInterrupt: 

In [44]:
def buscarmax(lista):
    i=0
    maxx=0
    for x in range(len(lista)):
        if (lista[x]>maxx):
            maxx=lista[x]
            i=x
    print(i,maxx)
    return i
mejor= buscarmax(Genetic1.Evaluation)

verjugar(Genetic1.Population[mejor])

17 256.0
ss
[[ 0.  2.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
1
[[  0.   2.   0.]
 [  0.  32.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
2
[[  0.   2.   0.]
 [  0.  64.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
3
[[  0.   2.   0.]
 [  0.  64.   0.]
 [  0.  16.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
4
[[  0.   2.  16.]
 [  0.  64.   0.]
 [  0.  16.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
5
[[  0.   2.  16.]
 [  0.  64.   2.]
 [  0.  16.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
6
[[  0.   2.  16.]
 [  0.  64.   4.]
 [  0.  16.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]]
7
[[  0.   2.  16.]
 [  0.  64.   4.]
 [  0.   0.  32.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.

 Redes Iniciales--------Generaciones-------------Mejor Jugador--------------------MejorJuego
       500                   100                      43                             32   en 10 juegos
       500                   200                      50                             35   en 10 juegos  
       500                   400                      59                             33   en 10 juegos 
       1000                  500                      85                             51   en 10 juegos 
       1000                  700                      77                             56   en 10 juegos 
       500                   400                     17 y 256                        33   en 10 juegos 
       1000              200 200                      58  y 384                      50  41        